In [2]:
%load_ext autoreload
%autoreload 2

import typing
from typing import List, Iterable
import pickle

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import rdkit
from rdkit.Chem import AllChem, Descriptors, MolFromSmiles

from utils import *

import os, glob

import gpytorch
import torch

from gauche.kernels.fingerprint_kernels.tanimoto_kernel import TanimotoKernel

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Loaded 97 descriptor functions
cuda:0


In [11]:
print(1, end='\r')
print(2, end='\r')

In [34]:
import gc
torch.cuda.empty_cache()
gc.collect()

1190

In [35]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
print((t-a)/1024/1024/1024)

7.729490280151367


In [3]:
with open("dataset/596/data.pkl", 'rb') as file:
    data = pickle.load(file)

X = np.array(data['fp_ls'])
y = np.array(data['activity_ls'])

X = X[:2000]
y = y[:2000]
X = X.astype(float)
y = y.reshape(-1,1).astype(float)

X = torch.from_numpy(X).type(torch.float16).to(device)
y = torch.from_numpy(y).type(torch.float16).flatten().to(device)

In [4]:
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import UnwhitenedVariationalStrategy


class GPClassificationModel(ApproximateGP):
    def __init__(self, train_x):
        variational_distribution = CholeskyVariationalDistribution(train_x.size(0))
        variational_strategy = UnwhitenedVariationalStrategy(
            self, train_x, variational_distribution, learn_inducing_locations=False
        )
        super(GPClassificationModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(TanimotoKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        latent_pred = gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        return latent_pred


# Initialize model and likelihood
model = GPClassificationModel(X)
likelihood = gpytorch.likelihoods.BernoulliLikelihood()

#
#likelihood = gpytorch.likelihoods.BetaLikelihood()

model.cuda()
likelihood.cuda()

BernoulliLikelihood(
  (quadrature): GaussHermiteQuadrature1D()
)

In [6]:
training_iterations = 100

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# "Loss" for GPs - the marginal log likelihood
# num_data refers to the number of training datapoints
mll = gpytorch.mlls.VariationalELBO(likelihood, model, y.numel())

for i in range(training_iterations):
    # Zero backpropped gradients from previous iteration
    optimizer.zero_grad()
    # Get predictive output
    output = model(X)

    print(output.shape)
    print(y.shape)
 
    # Calc loss and backprop gradients
    loss = -mll(output, y)
    loss.backward()
    if i%10 == 9:
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
    optimizer.step()

<bound method TorchDistributionMixin.shape of MultivariateNormal(loc: torch.Size([2000]))>
torch.Size([2000])
<bound method TorchDistributionMixin.shape of MultivariateNormal(loc: torch.Size([2000]))>
torch.Size([2000])
<bound method TorchDistributionMixin.shape of MultivariateNormal(loc: torch.Size([2000]))>
torch.Size([2000])
<bound method TorchDistributionMixin.shape of MultivariateNormal(loc: torch.Size([2000]))>
torch.Size([2000])
<bound method TorchDistributionMixin.shape of MultivariateNormal(loc: torch.Size([2000]))>
torch.Size([2000])
<bound method TorchDistributionMixin.shape of MultivariateNormal(loc: torch.Size([2000]))>
torch.Size([2000])
<bound method TorchDistributionMixin.shape of MultivariateNormal(loc: torch.Size([2000]))>
torch.Size([2000])
<bound method TorchDistributionMixin.shape of MultivariateNormal(loc: torch.Size([2000]))>
torch.Size([2000])


KeyboardInterrupt: 

In [10]:
model(X).sample()

tensor([ 0.5353,  0.0290,  0.7745,  ...,  1.3258, -1.1526, -0.7138],
       device='cuda:0')

In [37]:
f_preds = model(X)
y_preds = likelihood(model(X))

f_mean = f_preds.mean
f_samples = f_preds.sample(sample_shape=torch.Size((1000,)))

In [45]:
f_samples

tensor([[ 1.3138,  0.2986, -0.0682,  ...,  0.2259, -2.5279, -0.6808],
        [ 0.6883, -0.2564,  0.6097,  ...,  0.3862, -0.7299, -0.1721],
        [ 0.1128, -0.3425,  0.0262,  ...,  0.8339, -1.5077, -0.6605],
        ...,
        [ 1.7282,  0.1455,  0.1376,  ..., -1.1487, -0.7588, -0.9380],
        [ 0.2562, -0.6317,  1.1764,  ..., -0.7433, -0.8802,  0.0275],
        [ 0.5238,  0.3733,  1.0226,  ...,  0.3846, -0.9686,  0.4370]],
       device='cuda:0')

In [46]:
y_preds.sample()

tensor([1., 0., 0.,  ..., 0., 1., 0.], device='cuda:0')

In [11]:
y[model(X).sample().detach().cpu().numpy().argsort()].cpu().numpy()[:1000]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
       0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 1., 0., 1.